In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, BatchNormalization, Flatten, Dropout, RandomFlip, RandomRotation
from tensorflow.keras.layers import Rescaling, Resizing, AveragePooling2D, GlobalAvgPool2D
from tensorflow.keras.activations import relu
from tensorflow.keras.models import Model, Sequential
import numpy as np

In [3]:
from tensorflow.keras.applications import vgg16, vgg19, mobilenet, mobilenet_v2, mobilenet_v3

In [4]:
model = mobilenet_v2.MobileNetV2(include_top = False, input_shape = (224,224,3))
model.trainable = False

In [5]:
model_ = Sequential([
        model,
     GlobalAvgPool2D(),
    Flatten(),
    Dropout(0.8),
    Dense(102)
])

In [6]:
dataset_n = "oxford_flowers102"
import tensorflow_datasets as tfds

# Construct a tf.data.Dataset
ds_train, ds_test = tfds.load(dataset_n, split=['train','test'], shuffle_files=True, as_supervised = True)

In [7]:
def transform(image, label):
    image = Resizing(224,224)(image)
    image = Rescaling(1/127.5, offset = -1)(image)
    return image, label
def pipeline(data_set, train = True):
    data_set = data_set.map(transform, num_parallel_calls = tf.data.AUTOTUNE)
    data_set = data_set.cache()
    if train:
        data_set = data_set.shuffle(10000)
    data_set = data_set.batch(32)
    data_set = data_set.prefetch(tf.data.AUTOTUNE)
    return data_set   

In [8]:
ds_train = pipeline(ds_train)
ds_test = pipeline(ds_test, False)

In [10]:
model_.compile(optimizer= "adam", loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), metrics = tf.keras.metrics.SparseCategoricalAccuracy())
model_.fit(ds_train, epochs = 10, validation_data = ds_test) 

Epoch 1/10
32/32 [==============================] - 13s 326ms/step - loss: 1.3829 - sparse_categorical_accuracy: 0.6255 - val_loss: 1.2473 - val_sparse_categorical_accuracy: 0.7269
Epoch 2/10
32/32 [==============================] - 9s 302ms/step - loss: 1.1562 - sparse_categorical_accuracy: 0.6990 - val_loss: 1.1696 - val_sparse_categorical_accuracy: 0.7432
Epoch 3/10
32/32 [==============================] - 12s 374ms/step - loss: 1.0231 - sparse_categorical_accuracy: 0.7245 - val_loss: 1.1269 - val_sparse_categorical_accuracy: 0.7515
Epoch 4/10
32/32 [==============================] - 12s 375ms/step - loss: 0.8604 - sparse_categorical_accuracy: 0.7667 - val_loss: 1.0673 - val_sparse_categorical_accuracy: 0.7561
Epoch 5/10
32/32 [==============================] - 9s 298ms/step - loss: 0.8420 - sparse_categorical_accuracy: 0.7686 - val_loss: 1.0527 - val_sparse_categorical_accuracy: 0.7574
Epoch 6/10
32/32 [==============================] - 9s 300ms/step - loss: 0.7914 - sparse_categor

In [ ]:
y_t_pred =model_.predict()